<a href="https://colab.research.google.com/github/IRONSKULLGG/wild/blob/main/effnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import os

In [18]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elmadafri/the-wildfire-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/3


In [19]:
# Define the paths to your dataset
train_dir = r'/root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/3/the_wildfire_dataset_2n_version/train'
valid_dir = r'/root/.cache/kagglehub/datasets/elmadafri/the-wildfire-dataset/versions/3/the_wildfire_dataset_2n_version/val'

In [20]:
# Image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

In [21]:
# Data Augmentation and Image Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [22]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Binary classification (fire or no_fire)
)

Found 1887 images belonging to 2 classes.


In [24]:
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 402 images belonging to 2 classes.


In [25]:
# Load EfficientNetB0 model pre-trained on ImageNet
base_model = EfficientNetB0(include_top=False, input_shape=(img_height, img_width, 3))


Exception ignored in: <function AtomicFunction.__del__ at 0x7f260d421800>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 286, in __del__
    def __del__(self):
  
KeyboardInterrupt: 


In [26]:
# Freeze the base model layers
base_model.trainable = False

In [27]:
# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
steps_per_epoch = train_generator.samples // batch_size
validation_steps = valid_generator.samples // batch_size

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop if val_loss doesn't improve for 5 epochs
    restore_best_weights=True
)


history = model.fit(
   train_generator,
    validation_data=valid_generator,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)


Epoch 1/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 720s 12s/step - accuracy: 0.5427 - loss: 0.7201 - val_accuracy: 0.6120 - val_loss: 0.6680
Epoch 2/30
58/58 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.6562 - loss: 0.6369 - val_accuracy: 0.6172 - val_loss: 0.6654
Epoch 3/30
42/58 ━━━━━━━━━━━━━━━━━━━━ 2:33 10s/step - accuracy: 0.6274 - loss: 0.6677

In [15]:
    # Save the model
model.save('wildfire_detection_model.keras')



In [16]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(valid_generator)
print(f'Validation Accuracy: {val_acc}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 115s 9s/step - accuracy: 0.5718 - loss: 0.6844
Validation Accuracy: 0.611940324306488
